# Running Totals

This will calculate running totals of...
- Points
- Home Points
- Away Points
- Goal Difference
- Home Goal Difference
- Away Goal Difference

#### Import packages

In [1]:
import pandas as pd
import numpy as np
import time

pd.set_option('display.max_columns', None)

#### Read fixtures and teams file

In [2]:
fixtures = pd.read_csv('../../data/csv/fixtures.csv')
teams = pd.read_csv('../../data/csv/teams.csv')

In [3]:
# Check fixtures
fixtures.head()

,id,event,kickoff_time,team_h,team_a,team_h_score,team_a_score
0,0,1,2018-08-12 15:00:00+00:00,0,12,0,2
1,1,1,2018-08-11 14:00:00+00:00,1,4,2,0
2,2,1,2018-08-11 14:00:00+00:00,8,6,0,2
3,3,1,2018-08-11 14:00:00+00:00,9,5,0,3
4,4,1,2018-08-12 12:30:00+00:00,11,18,4,0


### Loop Information

This loop will look through all teams, and then their fixtures. Inside the middle loop, things calculated are...

- Points awarded
- Goal Difference

In [4]:
# Setup empty data frames

# Points
df_points = pd.DataFrame()
df_points_home = pd.DataFrame()
df_points_away = pd.DataFrame()
df_points_home_only = pd.DataFrame()
df_points_away_only = pd.DataFrame()

# Goal Difference
df_goal_difference = pd.DataFrame()
df_goal_difference_home = pd.DataFrame()
df_goal_difference_away = pd.DataFrame()
df_goal_difference_home_only = pd.DataFrame()
df_goal_difference_away_only = pd.DataFrame()

# Goals Scored
df_goals_scored = pd.DataFrame()
df_goals_scored_home = pd.DataFrame()
df_goals_scored_away = pd.DataFrame()
df_goals_scored_home_only = pd.DataFrame()
df_goals_scored_away_only = pd.DataFrame()

# Goals Conceded
df_goals_conceded = pd.DataFrame()
df_goals_conceded_home = pd.DataFrame()
df_goals_conceded_away = pd.DataFrame()
df_goals_conceded_home_only = pd.DataFrame()
df_goals_conceded_away_only = pd.DataFrame()

# Datetime
df_datetime = pd.DataFrame()

#### Start loop

In [5]:
# Start timing
start_time = time.time()

# Loop thorugh all 20 teams
for i in range(20):
    
    # empty points array
    points = np.zeros(38)
    points_home = np.zeros(38)
    points_away = np.zeros(38)
    points_home_only = np.zeros(38)
    points_away_only = np.zeros(38)
    
    # empty goals difference array
    goal_difference = np.zeros(38)
    goal_difference_home = np.zeros(38)
    goal_difference_away = np.zeros(38)
    
    # empty goals scored away
    goals_scored = np.zeros(38)
    goals_scored_home = np.zeros(38)
    goals_scored_away = np.zeros(38)
    
    # empty goals conceded
    goals_conceded = np.zeros(38)
    goals_conceded_home = np.zeros(38)
    goals_conceded_away = np.zeros(38)
    
    # datetime array
    datetime_array = [''] * 38
    
    # Emprt "was_home" array
    was_home = np.zeros(38, dtype='bool')
    
    # Get home and away fixtures
    a = np.logical_or(fixtures['team_h']==i, fixtures['team_a']==i)
    team_fixtures = (fixtures.loc[a,:]).reset_index(drop=True)
    team_fixtures['kickoff_time'] = pd.to_datetime(team_fixtures['kickoff_time'], utc=True)
    team_fixtures = team_fixtures.sort_values(by=['kickoff_time']).reset_index(drop=True)
    
    # loop thorugh fixtures. every team has 38 fixtures
    for j in range(38):
        
        # add datetime to array
        datetime_array[j] = team_fixtures['kickoff_time'][j]
        
        if team_fixtures['team_h'][j]==i:
            # if team is at home
            
            # was_home update
            was_home[j]=True
            
            # calculate poijts for team
            if team_fixtures['team_h_score'][j] > team_fixtures['team_a_score'][j]:
                points[j] = 3
            elif team_fixtures['team_h_score'][j] < team_fixtures['team_a_score'][j]:
                points[j] = 0
            else:
                points[j] = 1
                
            # set home and away points
            points_home[j] = points[j]
            points_away[j] = 0
            
            # goal difference
            goal_difference[j] = team_fixtures['team_h_score'][j] - team_fixtures['team_a_score'][j]
            goal_difference_home[j] = goal_difference[j]
            goal_difference_away[j] = 0
            
            # goals scored
            goals_scored[j] = team_fixtures['team_h_score'][j]
            goals_scored_home[j] = goals_scored[j]
            goals_scored_away[j] = 0
            
            # goals conceded
            goals_conceded[j] = team_fixtures['team_a_score'][j]
            goals_conceded_home[j] = goals_conceded[j]
            goals_conceded_away[j] = 0
            
        else:
            # if team is away
            if team_fixtures['team_h_score'][j] > team_fixtures['team_a_score'][j]:
                points[j] = 0
            elif team_fixtures['team_h_score'][j] < team_fixtures['team_a_score'][j]:
                points[j] = 3
            else:
                points[j] = 1
            
            # set home and away points
            points_away[j] = points[j]
            points_home[j] = 0
            
            # goal difference
            goal_difference[j] = team_fixtures['team_a_score'][j] - team_fixtures['team_h_score'][j]
            goal_difference_away[j] = goal_difference[j]
            goal_difference_home[j] = 0
            
            # goals scored
            goals_scored[j] = team_fixtures['team_a_score'][j]
            goals_scored_away[j] = goals_scored[j]
            goals_scored_home[j] = 0
            
            # goals conceded
            goals_conceded[j] = team_fixtures['team_h_score'][j]
            goals_conceded_away[j] = goals_conceded[j]
            goals_conceded_home[j] = 0
            
    
    # Points
    df_points[teams['short_name'][i]] = pd.Series(np.cumsum(points))
    df_points_home[teams['short_name'][i]] = pd.Series(np.cumsum(points_home))
    df_points_away[teams['short_name'][i]] = pd.Series(np.cumsum(points_away))
    df_points_home_only[teams['short_name'][i]] = pd.Series(np.cumsum(points_home[was_home]))
    df_points_away_only[teams['short_name'][i]] = pd.Series(np.cumsum(points_away[~was_home]))
        
    # Goal differnece
    df_goal_difference[teams['short_name'][i]] = pd.Series(np.cumsum(goal_difference))
    df_goal_difference_home[teams['short_name'][i]] = pd.Series(np.cumsum(goal_difference_home))
    df_goal_difference_away[teams['short_name'][i]] = pd.Series(np.cumsum(goal_difference_away))
    df_goal_difference_home_only[teams['short_name'][i]] = pd.Series(np.cumsum(goal_difference_home[was_home]))
    df_goal_difference_away_only[teams['short_name'][i]] = pd.Series(np.cumsum(goal_difference_away[~was_home]))
    
    # Goals scored
    df_goals_scored[teams['short_name'][i]] = pd.Series(np.cumsum(goals_scored))
    df_goals_scored_home[teams['short_name'][i]] = pd.Series(np.cumsum(goals_scored_home))
    df_goals_scored_away[teams['short_name'][i]] = pd.Series(np.cumsum(goals_scored_away))
    df_goals_scored_home_only[teams['short_name'][i]] = pd.Series(np.cumsum(goals_scored_home[was_home]))
    df_goals_scored_away_only[teams['short_name'][i]] = pd.Series(np.cumsum(goals_scored_away[~was_home]))
    
    # Goals conceded
    df_goals_conceded[teams['short_name'][i]] = pd.Series(np.cumsum(goals_conceded))
    df_goals_conceded_home[teams['short_name'][i]] = pd.Series(np.cumsum(goals_conceded_home))
    df_goals_conceded_away[teams['short_name'][i]] = pd.Series(np.cumsum(goals_conceded_away))
    df_goals_conceded_home_only[teams['short_name'][i]] = pd.Series(np.cumsum(goals_conceded_home[was_home]))
    df_goals_conceded_away_only[teams['short_name'][i]] = pd.Series(np.cumsum(goals_conceded_away[~was_home]))
    
    # Datetime
    df_datetime[teams['short_name'][i]] = pd.Series(datetime_array)
    
print("Time Taken: %s seconds" % (time.time() - start_time))

Time Taken: 0.31158995628356934 seconds


### Save all csvs

In [6]:
# Save all to csv

# Points
df_points.to_csv(r'../../data/csv/team_running_points.csv', index=False, index_label=False)
df_points_home.to_csv(r'../../data/csv/team_running_points_home.csv', index=False, index_label=False)
df_points_away.to_csv(r'../../data/csv/team_running_points_away.csv', index=False, index_label=False)

# Goal Difference
df_goal_difference.to_csv(r'../../data/csv/team_running_goal_difference.csv', index=False, index_label=False)
df_goal_difference_home.to_csv(r'../../data/csv/team_running_goal_difference_home.csv', index=False, index_label=False)
df_goal_difference_away.to_csv(r'../../data/csv/team_running_goal_difference_away.csv', index=False, index_label=False)

# Goals scored
df_goals_scored.to_csv(r'../../data/csv/team_running_goals_scored.csv', index=False, index_label=False)
df_goals_scored_home.to_csv(r'../../data/csv/team_running_goals_scored_home.csv', index=False, index_label=False)
df_goals_scored_away.to_csv(r'../../data/csv/team_running_goals_scored_away.csv', index=False, index_label=False)

# Goals conceded
df_goals_conceded.to_csv(r'../../data/csv/team_running_goals_conceded.csv', index=False, index_label=False)
df_goals_conceded_home.to_csv(r'../../data/csv/team_running_goals_conceded_home.csv', index=False, index_label=False)
df_goals_conceded_away.to_csv(r'../../data/csv/team_running_goals_conceded_away.csv', index=False, index_label=False)

# datetime
df_datetime.to_csv(r'../../data/csv/team_running_datetime.csv', index=False, index_label=False)

,ARS,BOU,BHA,BUR,CAR,CHE,CRY,EVE,FUL,HUD,LEI,LIV,MCI,MUN,NEW,SOU,TOT,WAT,WHU,WOL
0,0.0,3.0,0.0,1.0,0.0,3.0,3.0,1.0,0.0,0.0,0.0,3.0,3.0,3.0,0.0,1.0,3.0,3.0,0.0,1.0
1,0.0,6.0,3.0,1.0,1.0,6.0,3.0,4.0,0.0,0.0,3.0,6.0,6.0,3.0,1.0,1.0,6.0,6.0,0.0,1.0
2,3.0,7.0,3.0,1.0,2.0,9.0,3.0,5.0,3.0,1.0,6.0,9.0,7.0,3.0,1.0,1.0,9.0,9.0,0.0,2.0
3,6.0,7.0,4.0,1.0,2.0,12.0,3.0,6.0,4.0,2.0,6.0,12.0,10.0,6.0,1.0,4.0,9.0,12.0,0.0,5.0
4,9.0,10.0,5.0,1.0,2.0,15.0,6.0,6.0,4.0,2.0,6.0,15.0,13.0,9.0,1.0,5.0,9.0,12.0,3.0,8.0
5,12.0,10.0,5.0,4.0,2.0,16.0,7.0,6.0,5.0,2.0,9.0,18.0,16.0,10.0,2.0,5.0,12.0,13.0,4.0,9.0
6,15.0,13.0,5.0,7.0,2.0,17.0,7.0,9.0,5.0,2.0,12.0,19.0,19.0,10.0,2.0,5.0,15.0,13.0,7.0,12.0
7,18.0,16.0,8.0,8.0,2.0,20.0,7.0,12.0,5.0,3.0,12.0,20.0,20.0,13.0,2.0,5.0,18.0,13.0,7.0,15.0
8,21.0,17.0,11.0,8.0,5.0,21.0,7.0,15.0,5.0,3.0,12.0,23.0,23.0,14.0,2.0,6.0,21.0,16.0,7.0,15.0
9,22.0,20.0,14.0,8.0,5.0,24.0,8.0,15.0,5.0,3.0,13.0,26.0,26.0,17.0,3.0,7.0,21.0,19.0,8.0,15.0
